# Rappels jour 2 

* gotchas sur les fonctions 
    * pas d'arguments mutables en entré
    * accès aux varaible : L(ocal) E(nglobing) G(lobal) B(uiltin)
    * les variables englobing sont passées par closure
    * en python les closures : 
        * sont late binding 
        * sont en lecture seule
* curryfication =>  passer d'une fonction à N arguments à N fonctions imbriquées à 1 argument chacune 
* décorateurs
    * on peut le comprendre comme une composée de fonction `toto = deco(toto)` 
    * on peut chainer les décorateurs, l'ordre a une importance 
    * la fonction a l'intérieur s'appelle `wrapper` par convention 
    * dans le cas général, la bonne pratique consiste à décorer le wrapper avec `functools.wraps` pour conserver les métadonnées de la fonction d'origine (nom, docstring)
* les décorateurs peuvent être paramétrés, on rajoute une fonction qui va générer le décorateur qui nous intéresse

In [9]:
class Toto:
    def __init__(self, a):
        self.a = a 
    
    def hello(self):
        print(self.a)
    
    def __call__(self, b):
        print(b)
        
        
t = Toto("a")
t("b")

b
None


In [2]:
import inspect

In [3]:
inspect.getsourcefile(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [4]:
inspect.getsourcelines(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [23]:
from typing import Callable, TypeVar

T = TypeVar('T')

class MyDecorator:
    my_list = []
    
    def __init__(self, func: Callable[[T], T]):
        self.func = func
        self.my_list.append(func)
        
    def __call__(self, *args, **kwargs) -> T:
        print("Before function call")
        result = self.func(*args, **kwargs)
        print("After function call")
        return result
    
    def analyse(self):
        self.my_list

In [24]:
@MyDecorator
def my_function(arg: str) -> str:
    return arg.upper()


@MyDecorator
def toto(arg: str) -> str:
    return arg.upper()

my_function("hello")

Before function call
After function call


'HELLO'

In [25]:
MyDecorator.my_list

[<function __main__.my_function(arg: str) -> str>,
 <function __main__.toto(arg: str) -> str>]

# Gestion de fichiers / gestionnaires de contexte

In [54]:
try:
    f = open("./fibuzz.py")
except FileNotFoundError:
    print("le fichier n'est pas trouvé")
finally:
    try:
        f.close()
    except NameError:
        pass

le fichier n'est pas trouvé


In [49]:
f.closed

True

In [38]:
try:
    import numpi 
except ImportError as e:
    print(e, type(e))

numpi.array()

No module named 'numpi' <class 'ModuleNotFoundError'>


NameError: name 'numpi' is not defined

In [55]:
def exemple():
    try:
        print("dans try")
        return 1
    finally:
        return 2
    
exemple()

dans try


2

In [56]:
with open("./fizzbuzz.py") as f_text:
    1 / 0

ZeroDivisionError: division by zero

In [57]:
f_text.closed

True

In [67]:
class MonContextManager:
    def __enter__(self):
        print("dans enter")
        return "toto"

    def __exit__(self, exception_type, exception_object, traceback):
        print("dans exit", exception_type, exception_object, traceback)
    
mcm = MonContextManager()

with mcm as f:
    print("dans le with", f)

dans enter
dans le with toto
dans exit None None None


In [72]:
import time

class MonContextManager:
    def __enter__(self):
        self.tic = time.time()

    def __exit__(self, exception_type, exception_object, traceback):
        print(time.time() - self.tic)
    
mcm = MonContextManager()

with mcm as f:
    time.sleep(1)

1.0002660751342773


In [68]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")


with hello_context_manager() as hello:
    print(hello)


Entering the context...
Hello, World!
Leaving the context...


# Rappel matin

* fin des décorateurs
    * sous forme de classe => surcharger `__call__`
    * code à l'intérieur du wrapper éxécuté lors de l'appel de la fonction, code à l'extérieur exécuté au moment de la décoration de la fonction 
* context manager
    * à quoi ça sert ? isoler un environemment / un contexte dans un bloc, il se passe quelque chose à l'entrée et à la sortie du bloc 
    * restaurer un état même si le code à l'intérieur du bloc à crashé (fermer un fichier, relacher un verrou)
    * comment ça s'écrit ?
        * `with ... (as ...):`
        * implémenter sur n'importe quelle classe 2 méthodes : `__enter__` et `__exit__`
        * `@contextlib.contextmanager` => il faut qu'il y ait **un** `yield` (la fonction doit être un générateur), ce qui est avant sera dans le `__enter__` et après dans le `__exit__`
* POO
    * variables de classes
    * rappels POO

In [10]:
class Exemple:
    pass

e = Exemple()
print(e, type(e), hex(id(e)))

<__main__.Exemple object at 0x7fb4d40652e0> <class '__main__.Exemple'> 0x7fb4d40652e0


# A voir 

* copie de variables complexes